In [3]:
!pip install semopy networkx matplotlib pandas statsmodels dowhy --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 34.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 399.2/399.2 kB 25.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.0/193.0 kB 13.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 53.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 37.3/37.3 MB 20.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.2/100.2 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 222.1/222.1 kB 16.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 293.6/293.6 kB 19.3 MB/s eta 0:00:00


In [16]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
from statsmodels.tsa.stattools import grangercausalitytests
from semopy import Model
import warnings
warnings.filterwarnings("ignore")

# === 1. CARGAR DATOS INTERPOLADOS ===
# Asegúrate de que este archivo exista en tu carpeta content
data = pd.read_csv("/content/Datos1 - Interpolados.csv")

# Verifica las columnas
print("Columnas disponibles:", data.columns.tolist())

# Normalizar nombres
data.columns = [c.lower().strip() for c in data.columns]
if 'energia' in data.columns:
    data.rename(columns={'energia': 'energiapercapita'}, inplace=True)

# === 2. LIMPIEZA Y TRANSFORMACIÓN ===
vars_modelo = ['precipitacion', 'energiapercapita', 'produccion_agricola']
data = data[vars_modelo].dropna()

# Calcular diferencias porcentuales
data_diff = data.pct_change().dropna()
print("\nPrimeras filas de los datos transformados (% cambio):")
print(data_diff.head())

# === 3. TEST DE CAUSALIDAD DE GRANGER ===
max_lag = 1
granger_results = []

for x in ['precipitacion', 'energiapercapita']:
    y = 'produccion_agricola'
    try:
        test_result = grangercausalitytests(data_diff[[y, x]], max_lag, verbose=False)
        p_value = test_result[1][0]['ssr_ftest'][1]
        causalidad = p_value < 0.05
        granger_results.append({
            'Relacion': f"{x} → {y}",
            'p_value': round(p_value, 4),
            'Causalidad (α=0.05)': causalidad
        })
    except Exception as e:
        print(f"Error al probar {x} → {y}: {e}")
        granger_results.append({
            'Relacion': f"{x} → {y}",
            'p_value': np.nan,
            'Causalidad (α=0.05)': False
        })

granger_df = pd.DataFrame(granger_results)
print("\n=== RESULTADOS TEST DE GRANGER ===")
print(granger_df)

# === 4. MODELO DE ECUACIONES ESTRUCTURALES (SEM) ===
# Modelo teórico basado en conocimiento experto:
# Precipitacion y Energía influyen sobre Producción Agrícola

sem_model_desc = """
produccion_agricola ~ precipitacion + energiapercapita
"""

model = Model(sem_model_desc)
model.fit(data_diff)

# Extraer resultados como DataFrame
params_df = model.inspect(std_est=True)
print("\n=== RESULTADOS MODELO SEM ===")
print(params_df[['lval', 'op', 'rval', 'Estimate', 'Est. Std']]) # Corrected column name

# === 5. INTERPRETACIÓN AUTOMÁTICA ===
print("\n=== INTERPRETACIÓN ===")
for _, row in granger_df.iterrows():
    rel = row['Relacion']
    pv = row['p_value']
    if np.isnan(pv):
        print(f"{rel}: No se pudo calcular (posible ajuste perfecto o datos insuficientes).")
    elif pv < 0.05:
        print(f"{rel}: Existe evidencia estadísticamente significativa de causalidad temporal (p={pv}).")
    else:
        print(f"{rel}: No se detecta causalidad temporal significativa (p={pv}).")

print("\nEn el modelo SEM:")
for _, r in params_df.iterrows():
    if r['op'] == '~':  # solo relaciones de regresión
        signo = "positivo" if r['Estimate'] > 0 else "negativo"
        print(f"La variable '{r['rval']}' tiene un efecto {signo} sobre la producción agrícola ({r['Estimate']:.3f}).")

print("\n🧭 Conclusión general:")
print("Este análisis usa variaciones porcentuales para evitar correlaciones espurias.")
print("Si algún coeficiente o relación de Granger resulta significativa, puede considerarse evidencia causal preliminar.")
print("De lo contrario, podría tratarse de una relación inducida por interpolación o co-tendencia temporal.")

Columnas disponibles: ['anio', 'precipitacion', 'produccion_agricola', 'energiapercapita']

Primeras filas de los datos transformados (% cambio):
   precipitacion  energiapercapita  produccion_agricola
1      -0.076129         -0.033058             0.007590
2       0.208101         -0.034188             0.007533
3       2.699422          0.115044             0.007477
4      -0.714687          0.031746             0.007421
5       1.320372         -0.007692             0.007366

=== RESULTADOS TEST DE GRANGER ===
                                 Relacion  p_value  Causalidad (α=0.05)
0     precipitacion → produccion_agricola   0.8167                False
1  energiapercapita → produccion_agricola   0.0860                False

=== RESULTADOS MODELO SEM ===
                  lval  op                 rval      Estimate  Est. Std
0  produccion_agricola   ~        precipitacion      1.092363  0.006140
1  produccion_agricola   ~     energiapercapita      0.176017  0.000069
2  produccion_agric